In [ ]:
import os
import sys
import warnings
import numpy as np
from pathlib import Path
import numpy as np
import scipy.integrate
import sympy as sp
import pickle
import re
from tqdm import tqdm
import matplotlib.pyplot as plt

repo_path = Path("/content/odeformer")
if not repo_path.exists():
    !git clone https://github.com/sdascoli/odeformer.git {repo_path}

!pip install --quiet numexpr sympy==1.11.1 matplotlib numpy pandas requests scikit-learn scipy seaborn setproctitle torch tqdm wandb gdown regex
import torch
sys.path.append("/content/odeformer")
sys.path.append("/content/odeformer/envs")

sys.path.append("/content/odeformer/")
sys.path.append("/content/odeformer/odeformer")
sys.path.append("/content/odeformer/odeformer/envs")

Cloning into '/content/odeformer'...
remote: Enumerating objects: 1984, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 1984 (delta 200), reused 308 (delta 191), pack-reused 1652 (from 1)
Receiving objects: 100% (1984/1984), 40.82 MiB | 26.37 MiB/s, done.
Resolving deltas: 100% (1438/1438), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 763.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Try mounting again


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
samples_path = '/content/drive/MyDrive/aisc/samples_manual_1'
activations_path = '/content/drive/MyDrive/aisc/activations_manual_1'

In [ ]:
def clean_expression(expression):
    cleaned = expression.replace('--', '')
    cleaned = cleaned.replace(' -', '-')  # Remove space before negative sign
    cleaned = cleaned.replace('- ', '-')  # Remove space after negative sign
    return cleaned


In [ ]:
samples_path_old = '/content/drive/MyDrive/aisc/samples2'
samples_dir = os.fsencode(samples_path_old)
for sample in os.listdir(samples_dir):
  sample_name = os.fsdecode(sample)
  sample_path = os.path.join(samples_path_old, sample_name)
  test_sample = torch.load(sample_path)
  print(test_sample)
  break

{'times': [1.0, 1.183673469387755, 1.3673469387755102, 1.5510204081632653, 1.7346938775510203, 1.9183673469387754, 2.1020408163265305, 2.2857142857142856, 2.4693877551020407, 2.6530612244897958, 2.836734693877551, 3.0204081632653064, 3.2040816326530615, 3.3877551020408165, 3.5714285714285716, 3.7551020408163267, 3.938775510204082, 4.122448979591837, 4.3061224489795915, 4.4897959183673475, 4.673469387755102, 4.857142857142858, 5.040816326530613, 5.224489795918368, 5.408163265306123, 5.591836734693878, 5.775510204081633, 5.959183673469388, 6.142857142857143, 6.326530612244898, 6.510204081632653, 6.6938775510204085, 6.877551020408164, 7.061224489795919, 7.244897959183674, 7.428571428571429, 7.612244897959184, 7.795918367346939, 7.979591836734694, 8.16326530612245, 8.346938775510203, 8.53061224489796, 8.714285714285715, 8.89795918367347, 9.081632653061225, 9.26530612244898, 9.448979591836736, 9.63265306122449, 9.816326530612246, 10.0], 'trajectory': [-220264.65794806718, -1382383.563140076

In [ ]:
def generate_exp_decay_dictionaries(t, c_values, a_values):

    manual_samples = []

    for c_val in c_values:
        for a_val in a_values:
            trajectory = (c_val * np.exp(-a_val * t)).reshape(-1, 1)
            sample_dict = {
                'times': t,
                'trajectory': trajectory,
                'a': float(a_val),  # Convert to float for better serialization
                'c': float(c_val)   # Convert to float for better serialization
                ,'feature_dict': {"exponential_decay": 1, "Quadratic": 0}
                ,'expression': clean_expression(f"{c_val} * np.exp(-{a_val} * t)")
            }
            manual_samples.append(sample_dict)

    return manual_samples

In [ ]:
def generate_quadratic_dictionaries(t, c_values, t0_values):

    manual_samples = []

    for c_val in c_values:
        for t0_val in t0_values:
            trajectory = (c_val / (t0_val - t)).reshape(-1, 1)
            sample_dict = {
                'times': t,
                'trajectory': trajectory,
                'a': float(t0_val),  # Convert to float for better serialization
                'c': float(c_val)   # Convert to float for better serialization
                ,'feature_dict': {"exponential_decay": 0, "Quadratic": 1}
                ,'expression': clean_expression(f"{c_val} / ({t0_val} - t)")
            }
            manual_samples.append(sample_dict)

    return manual_samples

In [ ]:
t=np.linspace(1, 10, 50)
c = np.linspace(-10,10, 21)
a = np.linspace(-10,10, 21)

manual_samples = generate_exp_decay_dictionaries(t, c, a)
t0 = np.linspace(10.1, 100, 21)

manual_samples = manual_samples + generate_quadratic_dictionaries(t, c, t0)

In [ ]:
manual_samples[253]

{'times': array([ 1.        ,  1.18367347,  1.36734694,  1.55102041,  1.73469388,
         1.91836735,  2.10204082,  2.28571429,  2.46938776,  2.65306122,
         2.83673469,  3.02040816,  3.20408163,  3.3877551 ,  3.57142857,
         3.75510204,  3.93877551,  4.12244898,  4.30612245,  4.48979592,
         4.67346939,  4.85714286,  5.04081633,  5.2244898 ,  5.40816327,
         5.59183673,  5.7755102 ,  5.95918367,  6.14285714,  6.32653061,
         6.51020408,  6.69387755,  6.87755102,  7.06122449,  7.24489796,
         7.42857143,  7.6122449 ,  7.79591837,  7.97959184,  8.16326531,
         8.34693878,  8.53061224,  8.71428571,  8.89795918,  9.08163265,
         9.26530612,  9.44897959,  9.63265306,  9.81632653, 10.        ]),
 'trajectory': array([[1.62061679e+04],
        [8.46439069e+04],
        [4.42090384e+05],
        [2.30901331e+06],
        [1.20598472e+07],
        [6.29879063e+07],
        [3.28982306e+08],
        [1.71825616e+09],
        [8.97435566e+09],
        [4.

In [ ]:
i =0

for sample in manual_samples:

    print(sample['feature_dict'])
    sample_filename = f"sample_man_{i}.pt"
    #sample_filename = f"sample_man_{sample['expression']}.pt"
    sample_filepath = os.path.join(samples_path, sample_filename)
    i+=1
  # Save file using torch with pickle
    os.makedirs(os.path.dirname(sample_filepath), exist_ok=True)

    with open(sample_filepath, 'wb') as f:
      pickle.dump(sample, f)
    #torch.save(sample, sample_filepath)
    print(f"[INFO] Saved to {sample_filepath}")

{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_0.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_1.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_2.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_3.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_4.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_5.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_6.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to /content/drive/MyDrive/aisc/samples_manual_1/sample_man_7.pt
{'exponential_decay': 1, 'Quadratic': 0}
[INFO] Saved to

In [ ]:
from odeformer.model import SymbolicTransformerRegressor
dstr = SymbolicTransformerRegressor(from_pretrained=True)
model_args = {'beam_size': 50, 'beam_temperature': 0.1}
dstr.set_model_args(model_args)

Downloading...
From (original): https://drive.google.com/uc?id=1L_UZ0qgrBVkRuhg5j3BQoGxlvMk_Pm1W
From (redirected): https://drive.google.com/uc?id=1L_UZ0qgrBVkRuhg5j3BQoGxlvMk_Pm1W&confirm=t&uuid=2dc1ff48-6286-4679-9c83-56bd6bd50b32
To: /content/odeformer.pt
100%|██████████| 465M/465M [00:08<00:00, 57.7MB/s]


Loaded pretrained model


In [ ]:
layer_outputs = {}

# Function to store the output of each layer
def hook_fn(module, input, output, layer_name):
    layer_outputs[layer_name] = output.detach().cpu() #  detach to avoid unnecessary gradient tracking, and move to store in cpu

# Registering hooks for layers in the encoder and decoder
def register_hooks(model_part, part_name):
    for idx, module in enumerate(model_part.attentions):  # MultiHeadAttention layers
        layer_name = f"{part_name}_attention_{idx}"
        module.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))

    for idx, module in enumerate(model_part.ffns):  # FeedForward layers
        layer_name = f"{part_name}_ffn_{idx}"
        module.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))

    for idx, module in enumerate(model_part.layer_norm1):  # LayerNorm 1 layers
        layer_name = f"{part_name}_layer_norm1_{idx}"
        module.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))

    for idx, module in enumerate(model_part.layer_norm2):  # LayerNorm 2 layers
        layer_name = f"{part_name}_layer_norm2_{idx}"
        module.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))

# Registering hooks for the encoder and decoder parts
register_hooks(dstr.model.encoder, 'encoder')
register_hooks(dstr.model.decoder, 'decoder')

In [ ]:
os.makedirs(activations_path, exist_ok=True)

samples_dir = os.fsencode(samples_path)
for sample in os.listdir(samples_dir):
  sample_name = os.fsdecode(sample)
  sample_path = os.path.join(samples_path, sample_name)
  with open(sample_path, 'rb') as f:
    test_sample = pickle.load(f)
  #test_sample = torch.load(sample_path)
  print(f"[INFO] Loaded sample from {sample_path}")
  with torch.no_grad():
    dstr.fit(test_sample['times'], test_sample['trajectory'])
  encoder_layer_outputs = {}
  decoder_layer_outputs = {}
  activations = {}

  for layer_name, output in layer_outputs.items():
      if 'ffn' in layer_name:
      # Look at ouputs of ffn layers since they come before layer norm?
        if 'encoder' in layer_name:
          encoder_layer_outputs[layer_name] = output
        if 'decoder' in layer_name:
          decoder_layer_outputs[layer_name] = output

  activations['encoder'] = encoder_layer_outputs
  activations['decoder'] = decoder_layer_outputs
  try:
    activations['operator_dict'] = test_sample['operator_dict']
  except KeyError:
    pass  #
  activations['feature_dict'] = test_sample['feature_dict']

  test_seed = re.findall(r'\d+', sample_name)[0]
  activation_filename = f"activation_{test_seed}.pt"
  activation_filepath = os.path.join(activations_path, activation_filename)
  with open(activation_filepath, 'wb') as f:
    pickle.dump(activations, f)
  #torch.save(activations, activation_filepath)
  print(f"[INFO] Saved activations to {activation_filepath}")

[INFO] Loaded sample from /content/drive/MyDrive/aisc/samples_manual_1/sample_man_0.pt
[INFO] Saved activations to /content/drive/MyDrive/aisc/activations_manual_1/activation_0.pt
[INFO] Loaded sample from /content/drive/MyDrive/aisc/samples_manual_1/sample_man_1.pt
[INFO] Saved activations to /content/drive/MyDrive/aisc/activations_manual_1/activation_1.pt
[INFO] Loaded sample from /content/drive/MyDrive/aisc/samples_manual_1/sample_man_2.pt
[INFO] Saved activations to /content/drive/MyDrive/aisc/activations_manual_1/activation_2.pt
[INFO] Loaded sample from /content/drive/MyDrive/aisc/samples_manual_1/sample_man_3.pt
[INFO] Saved activations to /content/drive/MyDrive/aisc/activations_manual_1/activation_3.pt
[INFO] Loaded sample from /content/drive/MyDrive/aisc/samples_manual_1/sample_man_4.pt
[INFO] Saved activations to /content/drive/MyDrive/aisc/activations_manual_1/activation_4.pt
[INFO] Loaded sample from /content/drive/MyDrive/aisc/samples_manual_1/sample_man_5.pt
[INFO] Saved 

In [ ]:
print(activations['decoder'])

{'decoder_ffn_0': tensor([[[ 17.8524,  -2.6630, -12.6769,  ..., -22.3847,  17.2812,   3.9914]],

        [[ 17.0646,  -3.2149, -11.8618,  ..., -20.9452,  15.2709,   4.5102]],

        [[ 17.7785,  -2.7224, -12.8699,  ..., -22.2991,  16.9188,   4.2950]],

        ...,

        [[ 17.8524,  -2.6630, -12.6769,  ..., -22.3847,  17.2812,   3.9914]],

        [[ 17.8524,  -2.6630, -12.6769,  ..., -22.3847,  17.2812,   3.9914]],

        [[ 17.9511,  -2.6403, -13.0409,  ..., -22.7249,  17.3570,   3.2311]]]), 'decoder_ffn_1': tensor([[[  1.3206,  -4.3834, -21.5317,  ...,   0.8156,   1.9392,   9.9516]],

        [[  1.1768,  -4.5166, -22.2304,  ...,   0.6642,   2.0586,  10.3028]],

        [[  1.3261,  -4.4057, -21.6562,  ...,   0.8031,   1.9498,  10.0121]],

        ...,

        [[  1.3206,  -4.3834, -21.5317,  ...,   0.8156,   1.9392,   9.9516]],

        [[  1.3206,  -4.3834, -21.5317,  ...,   0.8156,   1.9392,   9.9516]],

        [[  1.3078,  -4.4154, -21.6249,  ...,   0.8212,   1.9587,  